# Import Packages

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

# Parameters

In [2]:
# Need input
input_file_name = "DSJC125.9_upper_Q"
file_type = ".txt"
file_path = ""

In [3]:
warm_start_file = ""
output_file_path = ""

In [5]:
warm_start = False
timeLimit = 1800
preQLin_val = 1
logToFile = 0    # 1 for logging to file or else 0

# Read Data

In [6]:
with open (file_path + input_file_name + file_type, "r") as file:
    count = 0
    for line in file:
        line = line.split()
        
        if (count == 2):
            n = int(line[2])
            K = int(line[3])
            p = int(line[4])
            Q = np.zeros((n*K,n*K))
        elif (count == 3):
            constant = int(line[4])
        elif (count >= 6):
            i = int(line[0])
            j = int(line[1])
            value = float(line[2])
            
            Q[i-1][j-1] = -value
                    
        count = count + 1

# Build Model

In [9]:
def get_last_best_value_and_array(file_path):
    # Initialize variables to hold the last "NEW BEST" value and array
    last_best_value = None
    last_best_array = []

    # Open the file
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Iterate over the lines in the file
    for i in range(len(lines)):
        # If a line contains "NEW BEST", store its value and the next line's array
        if "NEW BEST" in lines[i]:
            # Extract the value by splitting the line and converting to float
            last_best_value = float(lines[i].split(":")[2])
            
            # If the next line exists and is not an empty line
            if i+1 < len(lines) and lines[i+1].strip():
                # Convert the next line to an array of integers
                last_best_array = [int(char) for char in lines[i+1].strip() if char in '10']

    return last_best_value, last_best_array

In [ ]:
# Create a new model
m1 = Model("MSCP2")

# Logging
if logToFile == 1:
    m1.Params.LogFile = output_file_path + input_file_name + "_m3_"  + str(preQLin_val) + ".log"
    m1.Params.LogToConsole = 0

# Create variables
X = m1.addMVar(n*K, vtype=GRB.BINARY, name="X")

if warm_start:
    best_value, warm_start_values = get_last_best_value_and_array(warm_start_file)
    X.Start = warm_start_values

# Objective function
obj_func = X @ Q @ X + constant

# preprocessing type
m1.Params.PreQLinearize= preQLin_val

# Set objective
m1.setObjective(obj_func, GRB.MINIMIZE)

# Time limit
m1.Params.timelimit = timeLimit

# Optimize model
m1.optimize()

if logToFile == 1:
    m1.Params.LogFile = ""